In [1]:
import chess
import chess.pgn
import requests
import io
import json
from langchain_core.tools import tool
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage

In [2]:
%run ../bedrock_setup.py

In [3]:
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.vectorstores import FAISS

model_name = "sentence-transformers/all-MiniLM-L6-v2"
model_kwargs = {"device": "cuda"}
encode_kwargs = {"normalize_embeddings": True}
embeddings_model = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

vectorstore = FAISS.load_local("../database/knowledge_base", embeddings_model, allow_dangerous_deserialization=True)

/tmp/ipykernel_17736/2302536543.py:7: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings_model = HuggingFaceBgeEmbeddings(
/home/ec2-user/SageMaker/spring-2025-final-project-project-group-13/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from langchain.chat_models import init_chat_model

model = init_chat_model("us.anthropic.claude-3-5-haiku-20241022-v1:0",
                        model_provider="bedrock_converse",
                        region_name="us-east-1",
                        client=bedrock_client)

In [5]:
@tool
def query_knowledge_base(query: str) -> str:
    """Get relevant information on KB based on the query"""
    docs = vectorstore.similarity_search(query, k=10)
    return "\n\n".join([doc.page_content for doc in docs])

In [6]:
tools = [query_knowledge_base]

In [7]:
prompt = """
You are an intelligent agent that recommend chess moves to users.
Follow these steps to answer user queries:
1. Use the `query_knowledge_base` tool to get relevant information from the knowledge base based on the user query.

Use a step-by-step approach to complete each query.
"""

In [8]:
agent_executor = create_react_agent(model, tools, prompt = prompt)

In [9]:
query = "What are common opening moves?"
response = agent_executor.invoke({"messages": [HumanMessage(content=query)]})

In [10]:
response["messages"][-1].content

"Based on the knowledge base results, I'll provide an overview of common opening moves in chess:\n\n1. King's Pawn Opening (1.e4):\nThis is the most common opening move for White. The move 1.e4 is followed by Black often responding with 1...e5. This leads to several classic opening variations, including:\n- Two Knights Defense\n- King's Gambit\n- Open games with central pawn exchanges\n\n2. Queen's Pawn Opening (1.d4):\nAnother popular first move that aims to control the center of the board. This can lead to:\n- Various closed and semi-closed openings\n- Different pawn structures and game strategies\n\nKey Principles for Opening Moves:\n- Control the center of the board\n- Develop pieces quickly\n- Protect the king\n- Create opportunities for piece coordination\n\nSome common first move patterns include:\n- 1.e4 (King's Pawn)\n- 1.d4 (Queen's Pawn)\n- 1.c4 (English Opening)\n- 1.Nf3 (Reti Opening)\n\nThe most fundamental openings typically involve:\n- Moving central pawns (e4, d4, e5, 

In [11]:
docs = vectorstore.similarity_search(query, k=10)

In [12]:
docs

[Document(id='d15fbf13-7112-4af5-9dc7-cc369c7ae069', metadata={'type': 'opening-theory', 'title': 'An Opening Line For Club Players'}, page_content='five years ago, I wrote a series of articles "Openings For Tactical Players." The objective was to achieve an interesting, attacking position right out of the opening. And this is something most club players love. Unfortunately, some of those openings require a certain amount of opening theory and this is something many club players hate. But fortunately, there are some openings that lead to an entertaining play and also don\'t require almost any opening knowledge.One such opening, we discussedhere.Today, I want to talk about another opening line that could be played in your local club. Say you are playing a very solid opponent, who prefers to encounter your 1.e4 move with the French Defense. How can you get an attacking position there? Better yet, how to skip reading the opening books, checking the database and yet surprise your opening e